In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## In this notebook I use a public LGBM solution to demonstrate an idea for potentially a significant improvement. It revolves around the change in Rohlik sales with time. As we see from the data, in a lot of places, the sales grow on average (a more thorough look is needed). Which means just by looking at the store and day we can't really grasp the flow of time well, especially since the testing is conducted on a future data. So the decision tree models, that are generally not very good at extrapolating, will not give the best results

## The idea is to track how Rohlik sales generally change with time. For example, year-to-year. This can be further split for different regions etc. Like "here it grows 20% a year". Then we can include this observation in our models. What I mean is, if in June 2020 and September 2023 we had same sales, it doesnt mean in June 2024 and September 2024 it will be the same. And the model shouldn't aim at that. It should look at how particular days and other factors play a role, but considering the general trend of development of rohlik sales

## To demostrate it, in the most basic way, I multiply the predictions of the model by 1.05, since on average the sales grow, and the model, trained on past data, predicts somewhat less. It gives an improvement in public LB, and most likely in private even more so, since it has more recent data

## Which suggests it's good to try differrent ways to include the natural increase/decrease in sales with time, like the main trend. Because the date turns out to be very important, not only day, but also year, giving on average small (in some places significant) increments. But when the model is training, if it learns that in 2020 sales were lower, it wouldn't improve the score on test, because the data is much newer. So what we might want to try to extract from the train data, in addition to noticing particular good days / months / days of weeks, is how the sales change through the years

# Importing Libraries

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

# Importing Datasets

In [ ]:
train = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv', parse_dates=['date'])
test = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv', parse_dates=['date'])
ss = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv')
inventory = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv')
weights = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv')
calendar = pd.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv', parse_dates=['date'])

### Codeadd Markdown arrow_upwardarrow_downwarddelete Adding Missing Holidays taken from https://www.kaggle.com/code/yzokulu/rohlik-sales-forecasting-xgbregressor

In [ ]:
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]

budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

# Adding Features

In [ ]:
def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Brno_1'], holidays=brno_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Munich_1'], holidays=munich_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Frankfurt_1'], holidays=frank_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Budapest_1'], holidays=budapest_holidays)

In [ ]:
Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

In [ ]:
def process_calendar(df):
    df = df.sort_values('date').reset_index(drop=True)
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['weekday'] = df['date'].dt.weekday 
    return df

In [ ]:
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]
calendar_extended = pd.concat(processed_dfs).sort_values('date').reset_index(drop=True)

# Merging Datasets

In [ ]:
train_calendar = train.merge(calendar_extended, on=['date', 'warehouse'], how='left')
train_inventory = train_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')
train_data = train_inventory.merge(weights, on=['unique_id'], how='left')

test_calendar = test.merge(calendar_extended, on=['date', 'warehouse'], how='left')
test_datas = test_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')

# Droping Unwanted Columns

In [ ]:
train_data = train_data.drop(columns=['availability'])
train_data.dropna(subset=['sales'], inplace=True)

# Adding Extra Lag Features with respect to Date

In [ ]:
df = train_data
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['dayofweek'] = df['date'].dt.dayofweek
df['weekofyear'] = df['date'].dt.isocalendar().week
df['dayofyear'] = df['date'].dt.dayofyear
df['is_month_start'] = df['date'].dt.is_month_start
df['is_month_end'] = df['date'].dt.is_month_end
df['quarter'] = df['date'].dt.quarter
df["total_dic"] = df['type_0_discount'] + df['type_0_discount'] + df['type_1_discount'] + df['type_2_discount'] + df['type_3_discount'] + df['type_4_discount'] + df['type_5_discount'] + df['type_6_discount']
df['total_orders_'] = df['total_orders'] / df['sell_price_main']
df['total_orders_dic'] = df['total_orders_'] / df["total_dic"]
df['total_orders_sell_price_main'] = df['sell_price_main'] / df["total_dic"]

for i in range(7):
    df[f'total_orders{i}'] = df[f'type_{i}_discount'] / df["total_orders"]
    df[f'total_orders_sell_price_main_{i}'] = df[f'type_{i}_discount'] / df["total_orders_sell_price_main"]
    df[f'sell_price_main{i}'] = df[f'type_{i}_discount'] / df["sell_price_main"]
    df[f'sell_price_main_x_{i}'] = df[f'type_{i}_discount'] / (df["sell_price_main"] * df["total_orders"])
    df[f'total_orders_dic{i}'] = df[f'type_{i}_discount'] / df["total_orders_dic"]

    df[f'_total_orders{i}'] = df[f'type_{i}_discount'] * df["total_orders"]
    df[f'_total_orders_sell_price_main_{i}'] = df[f'type_{i}_discount'] * df["total_orders_sell_price_main"]
    df[f'_sell_price_main{i}'] = df[f'type_{i}_discount'] * df["sell_price_main"]
    df[f'_total_orders_dic{i}'] = df[f'type_{i}_discount'] * df["total_orders_dic"]

df.fillna(0, inplace=True)

# Rate of change of sales for each warehouse (year wise)

In [ ]:
# Group by warehouse and year to calculate total sales for each year
warehouse_yearly_sales = df.groupby(['warehouse', 'year'])['sales'].sum().reset_index()

# Calculate the year-to-year rise in sales and the percentage rise for each warehouse
warehouse_yearly_sales['sales_rise'] = warehouse_yearly_sales.groupby('warehouse')['sales'].diff()
warehouse_yearly_sales['sales_rise_percentage'] = warehouse_yearly_sales.groupby('warehouse')['sales'].pct_change() * 100

# Ensure there are no NaN values in the numeric columns
warehouse_yearly_sales['sales_rise'].fillna(0, inplace=True)
warehouse_yearly_sales['sales_rise_percentage'].fillna(0, inplace=True)

# Display the results
print(warehouse_yearly_sales)

# Integrated sales Change Multiplier

In [ ]:
import numpy as np

# Filter out the initial years where there's no previous year to compare
valid_sales_rise = warehouse_yearly_sales[warehouse_yearly_sales['year'] != 2020]

# Calculate the mean of the sales rise percentages, ignoring NaN values and infinities
mean_sales_rise_percentage = valid_sales_rise['sales_rise_percentage'].replace([np.inf, -np.inf], np.nan).mean(skipna=True)

# Convert percentage to a multiplier
sales_change_multiplier = 1 + (mean_sales_rise_percentage / 100)

# Ensure the multiplier is reasonable by bounding it within practical limits
sales_change_multiplier = max(min(sales_change_multiplier, 1.05), 1.0)

print("Integrated Sales Change Multiplier:", sales_change_multiplier)

In [ ]:
categorical_columns = df.select_dtypes("object").columns

for col in categorical_columns:
    df[col] = df[col].astype('category')

In [ ]:
df_test = test_datas
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day
df_test['weekday'] = df_test['date'].dt.weekday
df_test['dayofweek'] = df_test['date'].dt.dayofweek
df_test['weekofyear'] = df_test['date'].dt.isocalendar().week
df_test['dayofyear'] = df_test['date'].dt.dayofyear
df_test['is_month_start'] = df_test['date'].dt.is_month_start
df_test['is_month_end'] = df_test['date'].dt.is_month_end
df_test['quarter'] = df_test['date'].dt.quarter

df_test["total_dic"] = df_test['type_0_discount'] + df_test['type_0_discount'] + df_test['type_1_discount'] + df_test['type_2_discount'] + df_test['type_3_discount'] + df_test['type_4_discount'] + df_test['type_5_discount'] + df_test['type_6_discount']
df_test['total_orders_'] = df_test['total_orders'] / df_test['sell_price_main']
df_test['total_orders_dic'] = df_test['total_orders_'] / df_test["total_dic"]
df_test['total_orders_sell_price_main'] = df_test['sell_price_main'] / df_test["total_dic"]

for i in range(7):
    df_test[f'total_orders{i}'] = df_test[f'type_{i}_discount'] / df_test["total_orders"]
    df_test[f'total_orders_sell_price_main_{i}'] = df_test[f'type_{i}_discount'] / df_test["total_orders_sell_price_main"]
    df_test[f'sell_price_main{i}'] = df_test[f'type_{i}_discount'] / df_test["sell_price_main"]
    df_test[f'sell_price_main_x_{i}'] = df_test[f'type_{i}_discount'] / (df_test["sell_price_main"] * df_test["total_orders_sell_price_main"])
    df_test[f'total_orders_dic{i}'] = df_test[f'type_{i}_discount'] / df_test["total_orders_dic"]
    df_test[f'_total_orders{i}'] = df_test[f'type_{i}_discount'] * df_test["total_orders"]
    df_test[f'_total_orders_sell_price_main_{i}'] = df_test[f'type_{i}_discount'] * df_test["total_orders_sell_price_main"]
    df_test[f'_sell_price_main{i}'] = df_test[f'type_{i}_discount'] * df_test["sell_price_main"]
    df_test[f'_total_orders_dic{i}'] = df_test[f'type_{i}_discount'] * df_test["total_orders_dic"]

df_test.fillna(0, inplace=True)

for col in categorical_columns:
    df_test[col] = df_test[col].astype('category')

In [ ]:
train_start_date = '2020-08-01'
train_end_date = '2024-03-18'
test_start_date = '2024-03-18'
test_end_date = '2024-06-01'

X = df.drop(['sales', 'date', 'unique_id', 'weight'], axis=1)
y = np.sqrt(np.sqrt(df['sales']))

train_data = df[(df['date'] < train_end_date)]
test_data = df[(df['date'] >= test_start_date)]

X_train = train_data.drop(['sales', 'date', 'unique_id', 'weight'], axis=1)
y_train = np.sqrt(np.sqrt(train_data['sales']))
train_weights = train_data['weight']

X_test = test_data.drop(['sales', 'date', 'unique_id', 'weight'], axis=1)
y_test = np.sqrt(np.sqrt(test_data['sales']))
test_weights = test_data['weight']

In [ ]:
cols = X.select_dtypes(["int", "float"]).columns
categorical_feature_indices = [X.columns.get_loc(col) for col in categorical_columns if col in X.columns]

sc = StandardScaler()

for col in cols:
    X[col].replace([np.inf, -np.inf], X[col].min(), inplace=True)
    df_test[col].replace([np.inf, -np.inf], df_test[col].min(), inplace=True)

    X[col].fillna(X[col].mean(), inplace=True)
    df_test[col].fillna(df_test[col].mean(), inplace=True)

X[cols] = sc.fit_transform(X[cols])
df_test[cols] = sc.transform(df_test[cols])

# Boosting Using LightGBM

In [ ]:
from lightgbm import log_evaluation, early_stopping
callbacks = [log_evaluation(period=200)]

params = {
    'learning_rate': 0.021796506746095975,
    'num_leaves': 93,
    'max_depth': 10,
    'min_child_samples': 25,
    'subsample': 0.7057135664023435,
    'colsample_bytree': 0.8528497905459008,
    'reg_alpha': 0.036786449788597686,
    'reg_lambda': 0.3151110021900479,
    'num_boost_round': 9800,
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbose': -1
}

final_train_dataset = lgb.Dataset(X, label=y, 
                                  categorical_feature=categorical_feature_indices,
                                  weight=df['weight'])
final_model = lgb.train(params, 
                        final_train_dataset, 
                        num_boost_round=params['num_boost_round'],
                        callbacks=callbacks)
print('Model run successfully')

# Creating Submission File

In [ ]:
final_y_pred = final_model.predict(df_test.drop(['date', 'unique_id'], axis=1), num_iteration=final_model.best_iteration)
final_y_pred = np.power(np.power(final_y_pred, 2), 2)
sub = df_test.copy()
sub['sales_hat'] = final_y_pred * 1.05
sub['id'] = sub['unique_id'].astype(str) + "_" + sub['date'].astype(str)
sub[['id', 'sales_hat']].to_csv("submission.csv", index=False)

print('submission file created successfully')